# Optimization Example

## Setup

In [ ]:
import math

import matplotlib.pyplot as plt
from matplotlib.colors import PowerNorm
import numpy as np

plt.style.use(["notebook", {"image.cmap": "inferno"}])

In [ ]:
image_full = np.load("cas_a.npy").astype(np.float64)
image = image_full[300:500, 300:500]
print(image_full.shape, image.shape)

In [ ]:
plt.imshow(image_full, origin="lower", norm=PowerNorm(0.3))
plt.vlines([300,500],0,1000)
plt.hlines([300,500],0,1000)

In [ ]:
plt.imshow(image, origin="lower", norm=PowerNorm(0.3))

In [ ]:
def gaussian_kernel(sigma, size):
    """Create a 2D Gaussian kernel"""
    kernel = np.zeros((size, size))
    center = size // 2

    for i in range(size):
        for j in range(size):
            x = i - center
            y = j - center
            kernel[i, j] = math.exp(-(x**2 + y**2) / (2 * sigma**2))

    kernel = kernel / kernel.sum()
    return kernel

In [ ]:
kernel = gaussian_kernel(2.0, 11)
plt.imshow(kernel)

## Goal: Write function to smooth the image by the Gaussian kernel

This is just implementing a _Gaussian blur_, where $\hat I$ is the smoothed image with pixel values $\hat I_{i,j}$, and the source image is $I$, and the kernel is $K$, a smaller 2D array representing the kernel with pixel values $K_{m,n}$.  The $\circledast$ symbol represent the _convolution_ operator. 

$$
\begin{align*}
\hat I_{i,j}
=& (I \circledast K)_{i,j}\\
=& \sum_{m}\sum_{n} I_{i-m,\,j-n}\, K_{m,n}
\end{align*}
$$


In the actual implementation, we have to take into account some edge effects, so we add some padding 

## Use %timeit to measure performance

First we'll set up some useful functions, one to plot the result and one to compare performances.  

In [ ]:
def plot(before, after):
    """Plot the original and smoothed image"""
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    ax[0].imshow(before, origin="lower", norm=PowerNorm(0.2))
    ax[0].set_title("Original")

    ax[1].imshow(after, origin="lower", norm=PowerNorm(0.2))
    ax[1].set_title("Smoothed")


def plot_performance(perf: dict[str]):
    """Compare Performance between several timeit measurements."""
    names = list(perf.keys())
    averages = [m.average for m in perf.values()]
    std = [m.stdev for m in perf.values()]
    x = list(range(len(names)))

    plt.errorbar(x=x, y=averages, yerr=std, marker="o", linestyle="none")
    plt.xticks(x, names)
    plt.ylabel("average execution time (s)")
    plt.grid()

### A first implementation: python for-loops 

In [ ]:
def smooth_loops(image, kernel):
    """Smooth image with kernel, using simple python math."""
    h, w = image.shape
    kernel_size = kernel.shape[0]
    pad = kernel_size // 2
    result = np.zeros_like(image, dtype=float)

    for i in range(h):
        for j in range(w):
            value = 0.0
            for ki in range(kernel_size):
                for kj in range(kernel_size):
                    ii = i + ki - pad
                    jj = j + kj - pad
                    ii = max(0, min(h - 1, ii))
                    jj = max(0, min(w - 1, jj))
                    value += image[ii, jj] * kernel[ki, kj]

            result[i, j] = value

    return result

In [ ]:
image_smooth = smooth_loops(image, kernel)
plot(image, image_smooth)

To store my performance results, I create a dictionary called `PERF` that will hold the results by name. That's what the `plot_performance()` function expects as input

In [ ]:
PERF = dict()

And now we do the first measurement (note we use `%timeit -o` to get "output", which is an object containing some stats

In [ ]:
PERF["loops"] = %timeit -o smooth_loops(image, kernel)

In [ ]:
plot_performance(PERF)

### Second implementation: using NumPy to avoid some loops

In [ ]:
def smooth_numpy_basic(image, kernel):
    h, w = image.shape
    kernel_size = kernel.shape[0]
    pad = kernel_size // 2
    
    # Pad the image
    padded = np.pad(image, pad, mode='edge')
    result = np.zeros_like(image, dtype=float)
    
    for ki in range(kernel_size):
        for kj in range(kernel_size):
            result += padded[ki:ki+h, kj:kj+w] * kernel[ki, kj]
    
    return result

In [ ]:
image_smooth = smooth_numpy_basic(image, kernel)
plot(image, image_smooth)

In [ ]:
PERF['numpy basic'] = %timeit -o smooth_numpy_basic(image, kernel)

In [ ]:
plot_performance(PERF)

### Third implementation: Avoid all loops, advanced Numpy

In [ ]:
from numpy.lib.stride_tricks import as_strided


def smooth_numpy_noloops(image, kernel):
    h, w = image.shape
    kernel_size = kernel.shape[0]
    pad = kernel_size // 2
    
    # Pad the image
    padded = np.pad(image, pad, mode='edge')

    # Use as_strided to create sliding windows
    shape = (h, w, kernel_size, kernel_size)
    strides = padded.strides + padded.strides
    windows = as_strided(padded, shape=shape, strides=strides)
    
    # Multiply windows by kernel and sum
    result = np.sum(windows * kernel, axis=(2, 3))
    
    return result

In [ ]:
image_smooth = smooth_numpy_noloops(image, kernel)
plot(image, image_smooth)

In [ ]:
PERF["numpy noloop"] = %timeit -o smooth_numpy_noloops(image, kernel)

In [ ]:
plot_performance(PERF)
plt.yscale("log")

This actually is slower! And in fact uses way more memory!  (nice test for memory_profiler)

### Fourth Implementation: Using Numba to compile!

This is nice when you don't want to deal with numpy vectors and you prefer to keep it as loops:

In [ ]:
from numba import jit

@jit  # <--- the magic happens here
def smooth_loops_jit(image, kernel):
    """Smooth image with kernel, using simple python math."""
    h, w = image.shape
    kernel_size = kernel.shape[0]
    pad = kernel_size // 2
    result = np.zeros_like(image, dtype=np.float64)  # have to be careful of types though!

    for i in range(h):
        for j in range(w):
            value = 0.0
            for ki in range(kernel_size):
                for kj in range(kernel_size):
                    ii = i + ki - pad
                    jj = j + kj - pad
                    ii = max(0, min(h - 1, ii))
                    jj = max(0, min(w - 1, jj))
                    value += image[ii, jj] * kernel[ki, kj]

            result[i, j] = value

    return result

In [ ]:
image_smooth = smooth_loops_jit(image, kernel)
plot(image, image_smooth)

In [ ]:
PERF["loops jit"] = %timeit -o smooth_numpy_noloops(image, kernel)

In [ ]:
plot_performance(PERF)
plt.yscale("log")

### Fifth Implementation: use an existing library function

This is the simplest - we realize that scipy already implements this algorithm, and we can be pretty sure that they have done it in the most performant way, and this results in the simplest code, though it's also not easy to see what actually happens inside.

In [ ]:
from scipy.ndimage import convolve


def smooth_scipy(image, kernel):
    """Smooth image by kernel using a built-in scipy function."""
    return convolve(image, kernel, mode="nearest")

In [ ]:
image_smooth = smooth_scipy(image, kernel)
plot(image, image_smooth)

In [ ]:
PERF["scipy"] = %timeit -o smooth_scipy(image, kernel)

In [ ]:
plot_performance(PERF)
plt.yscale("log")

Also note, from `help(convolve)`:

> `convolve` has experimental support for **Python Array API Standard** compatible
backends in addition to NumPy. Please consider testing these features
by setting an environment variable ``SCIPY_ARRAY_API=1`` and providing
CuPy, PyTorch, JAX, or Dask arrays as array arguments. The following
combinations of backend and device (or other capability) are supported.

```
====================  ====================  ====================
Library               CPU                   GPU
====================  ====================  ====================
NumPy                 ✅                     n/a                 
CuPy                  n/a                   ✅                   
PyTorch               ✅                     ⛔                   
JAX                   ⚠️ no JIT             ⛔                   
Dask                  ⚠️ computes graph     n/a                 
====================  ====================  ====================
```

##  Function profiling:

There is a built-in "magic" function of Jupyter for running a cProfile profile on a statement.  See the lecture notes on how to run this outside of a notebook.

In [ ]:
%prun smooth_loops(image, kernel)

## Line Profiling

Looking at function calls doesn't give the full picture here. We just see that "min" and "max" are called many times . Let's see where the slowest part of `smooth_loops` is **by line**.

First we need to load the extension for Jupyter:

In [ ]:
%load_ext line_profiler

Now, we can use the magic %lprun Jupyter function to run the line profiler on the function we want.  Note that you can also do this outside of a notebook, see the lecture notes.

In [ ]:
%lprun -f smooth_loops smooth_loops(image, kernel)

## Memory Profiling with memory_profiler

* Now let's see which uses more memory, to get better stats let's run on the full image (*much* larger)
* we won't run `smooth_loops` as it is too slow for this demo... (but try it yourself, it should allocate very little memory)

In [ ]:
MEM = dict()
MEM["numpy noloops"] = %memit -o smooth_numpy_noloops(image_full, kernel)
MEM["numpy basic"] = %memit -o smooth_numpy_basic(image_full, kernel)
MEM["scipy"] = %memit -o smooth_scipy(image_full, kernel)

In [ ]:
def plot_memory(mem: dict[str]):
    names = list(mem.keys())
    peak = [max(m.mem_usage) for m in mem.values()]
    x = list(range(len(names)))

    plt.scatter(x=x, y=peak, marker="o")
    plt.xticks(x, names)
    plt.ylabel("Memory Usage")
    plt.grid()

In [ ]:
plot_memory(MEM)

**Conclusion**: It's not always best to eliminate all loops! not always faster, and can use much more memory!


## Let's also try another package: **memray** 

**note will not work on Windows!**


In [ ]:
%load_ext memray

In [ ]:
%%memray_flamegraph --temporary-allocations
smooth_numpy_basic(image_full, kernel)

In [ ]:
%%memray_flamegraph --temporary-allocations
smooth_numpy_noloops(image_full, kernel)

In [ ]:
%memit smooth_numpy_basic(image, kernel)

In [ ]:
%load_ext memray

### A memory leak

In [ ]:
%%memray_flamegraph
import numpy as np

SOME_GLOBAL_STATE = []

def my_leaky_function():
    SOME_GLOBAL_STATE.append(np.ones((1000,1000)))

for ii in range(1000):
    my_leaky_function()